In [1]:
import os
print(os.getcwd())

C:\Users\minij\OneDrive\Escritorio\Master\Spring 2023\Development of improved prediction methods for B cell epitopes prediction\Data


In [2]:
from make_dataset import Discotope_Dataset
import numpy as np
import torch
import pandas as pd

C:\Users\minij\anaconda3\envs\special_course\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Experimental PDB dataset
#train_set = torch.load("data/processed/new/bp3_train_seqs_solved/dataset.pt")
#valid_set = torch.load("data/processed/new/bp3_valid_seqs_solved/dataset.pt")
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

test_set = torch.load("SN_DT_SEMA_test_filtered_solved/dataset.pt")
test_set_af2 = torch.load("SN_DT_SEMA_test_filtered_af2/dataset.pt")
print(f"Length of test dataset: {len(test_set)} PDBs")
print(f"Length of test dataset (af2): {len(test_set_af2)} PDBs")

# train_set = torch.load("bp3_train_seqs_solved/dataset.pt")
# print(f"Length of train dataset: {len(train_set)} PDBs")

Length of test dataset: 24 PDBs
Length of test dataset (af2): 24 PDBs


In [4]:
stats_dfs_test = []
for i in range(0, len(test_set), 1):
    sample = test_set[i]
    df_sample = sample['df_stats']
    if(df_sample['rsa'].isna().any() == True):
        df_sample['epitope'] = sample['y_arr'].astype(bool)
        stats_dfs_test.append(df_sample)

In [5]:
# Validation loading

val_set = torch.load("bp3_valid_seqs_solved/dataset.pt")
val_set_af2 = torch.load("bp3_valid_seqs_af2/dataset.pt")

In [13]:
id_pdb = []
id_af2 = []
for i in range(0, len(val_set), 1):
    sample_af2 = val_set_af2[i]
    sample_pdb = val_set[i]
    id_pdb.append(sample_af2['pdb_id'])
    id_af2.append(sample_af2['pdb_id'])

id_pdb == id_af2

True

In [56]:
# Validation loading

val_set = torch.load("bp3_valid_seqs_solved/dataset.pt")
val_set_af2 = torch.load("bp3_valid_seqs_af2/dataset.pt")
print(f"Length of validation dataset (solved): {len(val_set)} PDBs")
print(f"Length of validation dataset (af2): {len(val_set_af2)} PDBs")

stats_dfs_test = []
for i in range(0, len(val_set), 1):
    sample = val_set[i]
    df_sample = sample['df_stats']
    if(sample['df_stats']['rsa'].isna().any() == True):
        df_sample['epitope'] = sample['y_arr'].astype(bool)
        stats_dfs_test.append(df_sample)

Length of validation dataset (solved): 281 PDBs
Length of validation dataset (af2): 281 PDBs


In [7]:
# Load one sample
sample = test_set[3]
print(sample.keys())

dict_keys(['pdb_id', 'pdb_fp', 'X_arr', 'y_arr', 'df_stats', 'length', 'pLDDTs', 'RSAs', 'sequence_str', 'pdb_seq', 'feature_idxs', 'struc_type'])


In [17]:
sample['y_arr']

array([0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [9]:
print(f"Features: {sample['X_arr'].shape}")
print(f"feature_columns: {sample['feature_idxs']}")
print(f"targets: {sample['y_arr'].shape}")

Features: (419, 536)
feature_columns: {'IF_tensor': range(0, 512), 'sequence': range(512, 532), 'pLDDTs': range(532, 533), 'lengths': range(533, 534), 'struc_type': range(534, 535), 'RSAs': range(535, 536)}
targets: (419,)


In [10]:
# DataFrame for overview, but use the features above for training!
df_stats = sample["df_stats"]
df_stats

,pdb,idx,residue,res_idx,rsa,diam,gyrrad,pLDDTs,length,struc_type
0,6wjl_G,1,s,1,0.974138,114.031616,32.728819,100.0,419,0
1,6wjl_G,2,c,2,0.247985,114.031616,32.728819,100.0,419,0
2,6wjl_G,3,a,3,0.435171,114.031616,32.728819,100.0,419,0
3,6wjl_G,4,e,4,0.379443,114.031616,32.728819,100.0,419,0
4,6wjl_G,5,t,5,0.013140,114.031616,32.728819,100.0,419,0
...,...,...,...,...,...,...,...,...,...,...
414,6wjl_G,415,l,415,0.495708,114.031616,32.728819,100.0,419,0
415,6wjl_G,416,g,416,0.135044,114.031616,32.728819,100.0,419,0
416,6wjl_G,417,h,417,0.583213,114.031616,32.728819,100.0,419,0
417,6wjl_G,418,d,418,0.329285,114.031616,32.728819,100.0,419,0


In [7]:
# Stack all features and targets to one big array
X_all = np.concatenate([sample["X_arr"] for sample in test_set])
y_all = np.concatenate([sample["y_arr"] for sample in test_set])
print(f"X_all: {X_all.shape}, y_all {y_all.shape}")

X_all: (7042, 536), y_all (7042,)


In [8]:
# One-hot-encoding for amino acids sequence
X_arr_sample = sample['X_arr']
len(X_arr_sample[:,513]), X_arr_sample[:,512:532]

(47,
 array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0.],
     

In [9]:
# Training loading

train_set = torch.load("bp3_train_seqs_solved/dataset.pt")
train_set_af2 = torch.load("bp3_train_seqs_af2/dataset.pt")
print(f"Length of train dataset (solved): {len(train_set)} PDBs")
print(f"Length of train dataset (af2): {len(train_set_af2)} PDBs")

Length of train dataset (solved): 1123 PDBs
Length of train dataset (af2): 1125 PDBs


In [10]:
# Counting total number of rows in the training data set

train_objects = 0
for i in range(len(train_set)):
    sample_ = train_set[i]
    train_objects = train_objects + sample_['X_arr'].shape[0]

train_af2_objects = 0
for i in range(len(train_set_af2)):
    sample_ = train_set_af2[i]
    train_af2_objects = train_af2_objects + sample_['X_arr'].shape[0]    

print(f"Number of rows from the train dataset (solved): {train_objects} instances")
print(f"Number of rows from the train dataset (af2): {train_af2_objects} instances")

Number of rows from the train dataset (solved): 240092 instances
Number of rows from the train dataset (af2): 240205 instances


In [11]:
print(f"Features: {sample_['X_arr'].shape}")
print(f"feature_columns: {sample_['feature_idxs']}")
print(f"targets: {sample_['y_arr'].shape}")

Features: (114, 536)
feature_columns: {'IF_tensor': range(0, 512), 'sequence': range(512, 532), 'pLDDTs': range(532, 533), 'lengths': range(533, 534), 'struc_type': range(534, 535), 'RSAs': range(535, 536)}
targets: (114,)


In [12]:
# Validation loading

val_set = torch.load("bp3_valid_seqs_solved/dataset.pt")
val_set_af2 = torch.load("bp3_valid_seqs_af2/dataset.pt")
print(f"Length of validation dataset (solved): {len(val_set)} PDBs")
print(f"Length of validation dataset (af2): {len(val_set_af2)} PDBs")

Length of validation dataset (solved): 281 PDBs
Length of validation dataset (af2): 281 PDBs


In [13]:
# Counting total number of rows in the validation data set

val_objects = 0
for i in range(len(val_set)):
    sample_ = val_set[i]
    val_objects = val_objects + sample_['X_arr'].shape[0]

val_af2_objects = 0
for i in range(len(val_set_af2)):
    sample_ = val_set_af2[i]
    val_af2_objects = val_af2_objects + sample_['X_arr'].shape[0]    

print(f"Number of rows from the validation dataset (solved): {val_objects} instances")
print(f"Number of rows from the validation dataset (af2): {val_af2_objects} instances")

Number of rows from the validation dataset (solved): 59896 instances
Number of rows from the validation dataset (af2): 59896 instances


In [14]:
print(f"Features: {sample_['X_arr'].shape}")
print(f"feature_columns: {sample_['feature_idxs']}")
print(f"targets: {sample_['y_arr'].shape}")

Features: (108, 536)
feature_columns: {'IF_tensor': range(0, 512), 'sequence': range(512, 532), 'pLDDTs': range(532, 533), 'lengths': range(533, 534), 'struc_type': range(534, 535), 'RSAs': range(535, 536)}
targets: (108,)


In [15]:
# Counting total number of rows in the test data set

test_objects = 0
for i in range(len(test_set)):
    sample_ = test_set[i]
    test_objects = test_objects + sample_['X_arr'].shape[0]

test_af2_objects = 0
for i in range(len(test_set_af2)):
    sample_ = test_set_af2[i]
    test_af2_objects = test_af2_objects + sample_['X_arr'].shape[0]    

print(f"Number of rows from the testidation dataset (solved): {test_objects} instances")
print(f"Number of rows from the testidation dataset (af2): {test_af2_objects} instances")

Number of rows from the testidation dataset (solved): 7042 instances
Number of rows from the testidation dataset (af2): 7042 instances
